# AIカメラを使ったじゃんけんゲーム（２）

AIカメラの学習が終了したのでPythonからAIカメラを制御します。まずAIカメラと接続するため以下のコードを書いてください。前回の顔認識と同じです。AIカメラを使うために必要な準備を行います。カメラとの接続はポートFを使ってください。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

次にAIカメラの動作を「Object Classification（モノの分類）」モードに切り替えるため、以下のコードを書いてください。

In [ ]:
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)

グーチョキパーが正しく識別できているか、以下のプログラムで確認しましょう

In [ ]:
while True:
    block = husky.read_blocks()
    print(block)

ここまででプログラムの全文は以下です

In [ ]:
#
# test program for AI camera
# mode: Object classification
#

import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)


while True:
    block = husky.read_blocks()
    print(block)


背景だけの場合は以下の出力になっていると思います。
```
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
[('block', 1, 160, 112, 224, 224)]
```
各リストの中の２番目の値が1になっています。これは背景がID1として識別されたことを表しています。最初に背景を学習させましたのでIDは１で合っています。

次に、じゃんけんのグーチョキパーを出した時にAIカメラが正しく識別するか確認しましょう。グーを出した時は以下の出力になりましたか？。blockの次の数字が2です。ID2としてグーを識別しています。背景の次にグーを学習させましたのでID2で合っています。

```
[('block', 2, 160, 112, 224, 224)]
[('block', 2, 160, 112, 224, 224)]
[('block', 2, 160, 112, 224, 224)]
[('block', 2, 160, 112, 224, 224)]
```
同様に、チョキ、パーも正しく識別されるかを確認しましょう。

実際にじゃんけんするには、'block'の次の数字(1,2,3)を取り出す必要があります。AIカメラから返却される値は以下の形式になっています。

<img src="https://github.com/TomoniCodeAcademy/LEGO-expoert-course/blob/main/img/document_parts/huskylens_data_format_object_classification.png?raw=true" width=400>

モノのIDを取り出すにはリストの先頭の値を取り出して、次に、リストの２番目の値を取り出す必要があります。Pythonで書くと以下のプログラムになります。

In [ ]:
aite = block[0][1]

上記のコードの意味は、まず配列の最初の値を取り出し(block[0])、取り出した値の2番目の値を取り出す(block[0][1])という意味です。２番目の値なのに[1]を指定しているのは、配列の位置は０番目から始まるためです（０スタートと呼びます）。この操作によりIDの番号が得られます。AIカメラから出力されたIDを正しく処理できているか、条件文を追加して確認しましょう

In [ ]:
while True:                      # ここは同じ
    block = husky.read_blocks()  # ここは同じ
    print(block)                 # ここは同じ
    # 以下を追加しましょう
    te = block[0][1]
    if te == 0:
        print('不明なモノ')
    elif te == 1:
        print('背景のみ')
    elif te == 2:
            print('グー')
    elif te == 3:
            print('チョキ')
    elif te == 4:
            print('パー')


:正しく、グー、チョキ、パーが出力されましたか？

以下のように表示されたら正しく処理でいています。もしグーを出しているのにパーと表示される場合は、IF文の条件が正しいか、IDが正しいか確認してください。IDが間違っている場合は、AIカメラの問題ですので、もう一度学習させてください。
```
背景のみ
[('block', 1, 160, 112, 224, 224)]
背景のみ
[('block', 1, 160, 112, 224, 224)]
背景のみ
[('block', 0, 160, 112, 224, 224)]
不明なモノ
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 0, 160, 112, 224, 224)]
不明なモノ
[('block', 3, 160, 112, 224, 224)]
チョキ
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 2, 160, 112, 224, 224)]
グー
[('block', 2, 160, 112, 224, 224)]
グー
```

LEGOとじゃんけん対決するには、LEGO側も、グー、チョキ、パーを出す必要があります。LEGOと対決するには、どれを出すか？をLEGOに決めさせる必要があります。LEGOが自分で出す手を決める方法として乱数を使う方法があります。Pythonでは乱数を発生させる関数、random.choice()というものがあり、これを使って、グーチョキパーを決めてランダムに出すことができます。LEGO側で手を決めるプログラムは以下です。

In [ ]:
import time
import random
while True:
    LEGO_te = random.choice(('guu','choki','paa'))
    print(LEGO_te)
    time.sleep(1)

time.sleep(1)はループが早すぎて見えないので１秒待ちを入れています。実行すると、以下の様に、LEGO側もじゃんけんをいろんな手を出してきます。
```
paa
choki
paa
choki
choki
guu
choki
choki
paa
guu
choki
```

ここまでのコードを整理します。全体をまとめると以下となります。先ほど作ったプログラムとくっつけてもwhile文があるため、この処理は実行できません。これは別プログラムファイルを作って実行してください。

In [ ]:
import sys
import time
import random
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

te = None
while True:
    # get te of LEGO
    LEGO_te = random.choice(('guu','choki','paa'))

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        te = block[0][1]
    print(te, LEGO_te)
    time.sleep(1)

上記プログラムを実行すると以下となります。確かに、それぞれが手を出していますが、人の手は1,2,3の数値で、LEGOの手は'guu', 'choki', 'paa'の文字列です。これでは比較ができません。数値による人の手をLEGOの手を同じ文字列にそろえる必要があります。そこで、数値->文字列に変換する関数を作ります。変換関数は以下です。

In [ ]:
# convert
# hand id to hand string
#  1: 'guu' , 2: 'choki' 3: 'paa'
def  hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

LEGOが選んだ、guu, choki, paa と人が選んだ guu , choki, paaを毎回表示するプログラムは以下となります

In [ ]:
import sys
import time
import random
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

# convert
# hand id to hand string
#  1: 'guu' , 2: 'choki' 3: 'paa'
def  hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

hand_str = None
while True:

    # get te of LEGO
    LEGO_choice = random.choice(('guu','choki','paa'))

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        hand_id = block[0][1]
        hand_str = hand_id2str(hand_id)
    print(hand_str, LEGO_choice)
    time.sleep(1)

上記プログラムの出力例は以下です。表示されているぐ、ちょき(guu choki)、のうち、左側が人で、右側がLEGOです。
```
guu paa
[('block', 2, 160, 112, 224, 224)]
guu paa
[('block', 3, 160, 112, 224, 224)]
choki choki
[('block', 4, 160, 112, 224, 224)]
paa choki
[('block', 0, 160, 112, 224, 224)]
unknown guu
[('block', 0, 160, 112, 224, 224)]
unknown choki
[('block', 0, 160, 112, 224, 224)]
unknown guu
[('block', 0, 160, 112, 224, 224)]
unknown guu
```
このプログラムで、LEGOのguu,choki,paaと、人の手による guu, choki, paa を並べて出力するところまでできました。
このままでは、各じゃんけんで、人が勝ったのか、LEGOが勝ったのか分かりません。そこで、じゃんけんの勝ち負け反転関数を作り、どちらが勝ったのかを表示します。プログラムは以下です。

In [ ]:
#
# hand1, hand2 が guu, choki, paa を選択した時、
# hand1側が勝ったのか負けたのか引き分けか？を返します
#
def judge_winner(hand1, hand2):
    if hand1 == 'guu':
        if hand2 == 'guu':
            return 'draw'
        elif hand2 == 'choki':
            return 'win'
        elif hand2 == 'paa':
            return 'lose'
    elif hand1 == 'choki':
        if hand2 == 'guu':
            return 'lose'
        elif hand2 == 'choki':
            return 'draw'
        elif hand2 == 'paa':
            return 'win'
    elif hand1 == 'paa':
        if hand2 == 'guu':
            return 'win'
        elif hand2 == 'choki':
            return 'lose'
        elif hand2 == 'paa':
            return 'draw'


先ほどのプログラムに組み込みます

In [ ]:
import sys
import time
import random
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

# convert
# hand id to hand string
#  1: 'guu' , 2: 'choki' 3: 'paa'
def  hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

#
# hand1, hand2 が guu, choki, paa を選択した時、
# hand1側が勝ったのか負けたのか引き分けか？を返します
#
def judge_winner(hand1, hand2):
    if hand1 == 'guu':
        if hand2 == 'guu':
            return 'draw'
        elif hand2 == 'choki':
            return 'win'
        elif hand2 == 'paa':
            return 'lose'
    elif hand1 == 'choki':
        if hand2 == 'guu':
            return 'lose'
        elif hand2 == 'choki':
            return 'draw'
        elif hand2 == 'paa':
            return 'win'
    elif hand1 == 'paa':
        if hand2 == 'guu':
            return 'win'
        elif hand2 == 'choki':
            return 'lose'
        elif hand2 == 'paa':
            return 'draw'

hand_str = None
while True:
    # get te of LEGO
    LEGO_choice = random.choice(('guu','choki','paa'))

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        hand_id = block[0][1]
        hand_str = hand_id2str(hand_id)
    result = judge_winner(hand_str, LEGO_choice)
    print(hand_str, LEGO_choice, 'you are..', result)
    time.sleep(1)

上記プログラムを実行すると、以下の出力となります。
ずっとグーを出していましたが、LEGO側がランダムにいろんな手を出していて、勝ち負けが正しく判定されていることが分かります。
```
[('block', 2, 160, 112, 224, 224)]
guu guu you are.. draw
[('block', 2, 160, 112, 224, 224)]
guu choki you are.. win
[('block', 2, 160, 112, 224, 224)]
guu choki you are.. win
[('block', 2, 160, 112, 224, 224)]
guu choki you are.. win
[('block', 2, 160, 112, 224, 224)]
guu paa you are.. lose
[('block', 0, 160, 112, 224, 224)]
unknown choki you are.. None
[('block', 3, 160, 112, 224, 224)]
choki choki you are.. draw
[('block', 1, 160, 112, 224, 224)］
```

ここまでで、人の手（グーチョキパー）とLEGOの手でじゃんけんをして、勝ち負けの判定までできました。

ですが、これでは一体何をやっているのか分かりませんし、ゲームらしくありません。次に、音を鳴らしてリズムを取ったり、勝ち負けに応じて、歓声が湧いたりするように改良しましょう。